In [42]:
import pandas 

zip_codes_by_city = pandas.read_csv("./free-zipcode-database-Primary.csv")
new_york_zip_codes = zip_codes_by_city[zip_codes_by_city["City"].isin(["BRONX", "BROOKLYN","NEW YORK","STATEN ISLAND", "QUEENS"])]
time_square_zip_code = 10036
map_center_info = new_york_zip_codes[new_york_zip_codes["Zipcode"]==time_square_zip_code]
new_york_zip_codes["geolocation"]= list(zip(new_york_zip_codes.loc[:,"Lat"], new_york_zip_codes.loc[:,"Long"]))
zip_code_locations = new_york_zip_codes["geolocation"].tolist()

/Users/ianborukhovich/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [43]:
import gmaps
import gmaps.datasets

api_key = open("../shared/shared_api_key.txt").readline()
gmaps.configure(api_key=api_key) # Your Google API key

# load a Numpy array of (latitude, longitude) pairs
data = gmaps.datasets.load_dataset('taxi_rides')
m = gmaps.Map()
m.add_layer(gmaps.Heatmap(data=data))
m

In [44]:
m = gmaps.Map()


In [22]:
import requestsimport requests
import json
#response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=["1510 e 9th street"]&sensor=false')
#json_data = json.loads(response.text)


,Zipcode,ZipCodeType,City,State,LocationType,Lat,Long,Location,Decommisioned,TaxReturnsFiled,EstimatedPopulation,TotalWages
2951,10036,STANDARD,NEW YORK,NY,PRIMARY,40.71,-73.99,NA-US-NY-NEW YORK,False,17030.0,21944.0,1.686575e+09


In [38]:
new_york_zip_codes["geolocation"].tolist()

[(40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000000000005),
 (40.840000000000003, -73.870000